In [2]:
# initialize stuff
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import pickle
import importlib
import logging

import numpy as np
import matplotlib.pyplot as plt

sys.path.append("/Users/oliver/Documents/p5control-bluefors-evaluation")
sys.path.append("/home/oliver/Documents/p5control-bluefors-evaluation")

from utilities.baseclass import BaseClass
from utilities.baseevaluation import BaseEvaluation
from utilities.baseplot import BasePlot
from utilities.ivevaluation import IVEvaluation
from utilities.basefunctions import bin_y_over_x

importlib.reload(sys.modules["utilities.baseclass"])
importlib.reload(sys.modules["utilities.baseevaluation"])
importlib.reload(sys.modules["utilities.baseplot"])
importlib.reload(sys.modules["utilities.ivevaluation"])

import utilities.basefunctions
importlib.reload(sys.modules["utilities.basefunctions"])

from utilities.corporate_design_colors_v4 import cmap
importlib.reload(sys.modules["utilities.corporate_design_colors_v4"])

importlib.reload(logging)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(message)s")
# pip install numpy matplotlib scipy tqdm torch hdf5view h5py PySide6


In [5]:
path = 'theoretical IVs/CarlosIV/'
arr = os.listdir(f'{path}')
arr.remove('iv.in')
arr.remove('iv.for')

v_norm = np.linspace(0.01,5.99,599)
V_norm = np.concatenate((-np.flip(v_norm),v_norm))
T_theo = np.zeros(len(arr)+1, dtype='float64')
I_norm_theo = np.zeros((len(T_theo), len(v_norm)), dtype='float64')

for i in range(len(arr)):
    temp = arr[i][:-4]
    value = int(temp)*10**(-len(temp)+1)
    T_theo[i+1] = value

for i, a in enumerate(arr):
    v_norm_theo = np.genfromtxt(f'{path}/{a}')[:,0]
    i_norm_theo = np.genfromtxt(f'{path}/{a}')[:,1]
    I_norm_theo[i+1,:], _ = bin_y_over_x(v_norm_theo, i_norm_theo, v_norm)

I_theo = np.concatenate((-np.flip(I_norm_theo, axis=1),I_norm_theo), axis=1)

dIdV_theo = np.gradient(I_theo, V_norm, axis=1)

carlos = {
    'voltage': V_norm,
    'transmission': T_theo,
    'current': I_theo,
    'dIdV': dIdV_theo,
}

indices = np.argsort(-carlos['transmission'])
theo = dict(
    voltage = carlos['voltage'],
    transmission = carlos['transmission'][indices],
    current = carlos['current'][indices, :],
    dIdV = carlos['dIdV'][indices, :],
    )

with open('carlosIV.pickle', 'wb') as f:
    pickle.dump(theo, f)
